# Create VRT to bring RS back to map

In [1]:
import rasterio
from rasterio.windows import Window
from PIL import Image
import os
import csv

In [2]:
# Function to generate a VRT file for each image
def create_vrt_for_tile(output_folder, img_id, tile_width, tile_height, min_x, min_y, max_x, max_y, city):
    # Calculate pixel size in X and Y directions
    pixel_size_x = (max_x - min_x) / tile_width
    pixel_size_y = (min_y - max_y) / tile_height  # Note: Y values are typically decreasing in north-up images
    
    # VRT content template
    vrt_content = f"""
    <VRTDataset rasterXSize="{tile_width}" rasterYSize="{tile_height}">
      <SRS>EPSG:4326</SRS>
      <GeoTransform>{min_x}, {pixel_size_x}, 0, {max_y}, 0, {pixel_size_y}</GeoTransform>
      <VRTRasterBand dataType="Byte" band="1">
        <SimpleSource>
          <SourceFilename relativeToVRT="1">{img_id}.jpg</SourceFilename>
          <SourceBand>1</SourceBand>
          <SrcRect xOff="0" yOff="0" xSize="{tile_width}" ySize="{tile_height}"/>
          <DstRect xOff="0" yOff="0" xSize="{tile_width}" ySize="{tile_height}"/>
        </SimpleSource>
      </VRTRasterBand>
      <VRTRasterBand dataType="Byte" band="2">
        <SimpleSource>
          <SourceFilename relativeToVRT="1">{img_id}.jpg</SourceFilename>
          <SourceBand>2</SourceBand>
          <SrcRect xOff="0" yOff="0" xSize="{tile_width}" ySize="{tile_height}"/>
          <DstRect xOff="0" yOff="0" xSize="{tile_width}" ySize="{tile_height}"/>
        </SimpleSource>
      </VRTRasterBand>
      <VRTRasterBand dataType="Byte" band="3">
        <SimpleSource>
          <SourceFilename relativeToVRT="1">{img_id}.jpg</SourceFilename>
          <SourceBand>3</SourceBand>
          <SrcRect xOff="0" yOff="0" xSize="{tile_width}" ySize="{tile_height}"/>
          <DstRect xOff="0" yOff="0" xSize="{tile_width}" ySize="{tile_height}"/>
        </SimpleSource>
      </VRTRasterBand>
    </VRTDataset>
    """
    
    # Save the VRT file
    vrt_filename = os.path.join(output_folder, f'{img_id}.vrt')
    with open(vrt_filename, 'w') as vrt_file:
        vrt_file.write(vrt_content)
    
    # print(f"VRT file created for {img_id} at {vrt_filename}")

# Function to read the CSV and process each row
def process_csv_and_create_vrt(csv_file, output_folder):
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        
        for row in reader:
            img_id = row['img_id']
            tile_width = int(row['tile_width'])
            tile_height = int(row['tile_height'])
            min_x = float(row['x_coord'])  # Top-left X
            max_y = float(row['y_coord'])  # Top-left Y
            # Extract coordinates from the grid polygon (WKT)
            polygon_coords = row['grid_polygon'].replace('POLYGON((', '').replace('))', '').split(', ')
            bottom_left = polygon_coords[0].split()  # (min_x, min_y)
            min_y = float(bottom_left[1])  # Bottom Y
            top_right = polygon_coords[2].split()  # (max_x, max_y)
            max_x = float(top_right[0])  # Right X
            
            # Call the function to create VRT
            create_vrt_for_tile(output_folder, img_id, tile_width, tile_height, min_x, min_y, max_x, max_y, row['city'])
        print(f"VRT files created")

In [4]:
csv_file = '/Users/wenlanzhang/Downloads/15_13722_2676_大图/BJ_metadata.csv'  # Path to Img CSV file
output_folder = '/Users/wenlanzhang/Downloads/15_13722_2676_大图'  # Folder where VRT files should be saved

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process the CSV and create VRT files
process_csv_and_create_vrt(csv_file, output_folder)


VRT files created
